In [176]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline, Pipeline
import joblib
import difflib
import os
from sklearn.svm import SVC

In [218]:
# ==========================
# 1. Carregar o dataset
# ==========================
df = pd.read_excel("movies_database.xlsx", engine='openpyxl')


In [219]:
df['rating_imdb'] = df['rating_imdb'].astype(float)
df['year'] = df['year'].fillna(0).astype(int)
df['gross_world_wide'] = df['gross_world_wide'].astype(float)
df['budget'] = df['budget'].astype(float)
df['gross_opening_weekend'] = df['gross_opening_weekend'].astype(float)
df['oscar'] = df['oscar'].fillna(0).astype(int)
df['nomination'] = df['nomination'].fillna(0).astype(int)
df['duration'] = df['duration'].astype(str)

In [220]:
# Função para converter o tempo em minutos
def converter_tempo_para_minutos(tempo_str):
    if pd.isna(tempo_str) or tempo_str.lower() == 'nan':
        return None
    partes = tempo_str.replace('h', '').replace('m', '').split()
    try:
        horas = int(partes[0]) if partes[0] else 0
    except ValueError:
        horas = 0
    try:
        minutos = int(partes[1]) if len(partes) > 1 else 0
    except (ValueError, IndexError):
        minutos = 0
    return (horas * 60) + minutos

# Aplique a função à coluna 'duration' e crie uma nova coluna 'duration_min'
df['duration_min'] = df['duration'].apply(converter_tempo_para_minutos)

In [ ]:
# ==========================
# 2. Definir intents e exemplos de perguntas
# ==========================
perguntas_iniciais = [
    "Qual o filme mais bem avaliado?",
    "Qual o filme com maior bilheteira mundial?",
    "Quem é o diretor de Titanic?",
    "Quantos Oscars ganhou Avatar?",
    "Qual o gênero do filme Interstellar?",
    "Qual o filme com mais indicações ao Oscar?",
    "Qual o filme mais longo?",
    "Qual o filme mais recente?"
]

intencoes_iniciais = [
    "melhor_nota",
    "maior_bilheteria",
    "diretor_filme",
    "quantidade_oscar_filme",
    "genero_filme",
    "mais_indicacoes",
    "maior_duracao",
    "filme_recente"
]

In [ ]:
if os.path.exists("perguntas_aprendidas.csv"):
    base_aprendida = pd.read_csv("perguntas_aprendidas.csv")
else:
    # Criamos a base com os dados iniciais
    base_aprendida = pd.DataFrame({
        "pergunta": perguntas_iniciais,
        "intencao": intencoes_iniciais
    })

In [247]:
base_aprendida

,pergunta,intencao
0,Qual o filme mais bem avaliado?,melhor_nota
1,Qual o filme com maior bilheteira mundial?,maior_bilheteria
2,Quem é o diretor de Titanic?,diretor_filme
3,Quantos Oscars ganhou Avatar?,quantidade_oscar_filme
4,Qual o gênero do filme Interstellar?,genero_filme
5,Qual o filme com mais indicações ao Oscar?,mais_indicacoes
6,Qual o filme mais longo?,maior_duracao
7,Qual o filme mais recente?,filme_recente


In [248]:
# ==========================
# 2. Criar o modelo SVM com pipeline
# ==========================
# Criar o modelo de linguagem com TF-IDF e SVM
vectorizer = TfidfVectorizer()
modelo = make_pipeline(vectorizer, SVC(kernel='linear'))  # Usando SVM com kernel linear

In [249]:
# Vetorização das perguntas
modelo.fit(base_aprendida["pergunta"], base_aprendida["intencao"])  # Treinamento do modelo

,steps,"[('tfidfvectorizer', ...), ('svc', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [250]:
def salvar_nova_pergunta(pergunta, resposta, intencao):
    """Salva perguntas novas para aprendizado contínuo"""
    global base_aprendida

    if not pergunta in base_aprendida["pergunta"].values and pergunta.strip() != "":
        nova = pd.DataFrame([[pergunta, intencao, resposta]], columns=["pergunta", "intencao", "resposta"])
        base_aprendida = pd.concat([base_aprendida, nova], ignore_index=True)
        base_aprendida.to_csv("perguntas_aprendidas.csv", index=False)

    # Re-treinar o modelo com todos os dados
    modelo.fit(base_aprendida["pergunta"], base_aprendida["intencao"])

In [251]:
# ==========================
# 4. Funções auxiliares
# ==========================
def encontrar_filme(titulo):
    """Retorna o filme mais parecido com o nome fornecido"""
    titulos = [str(t) for t in df['title'].tolist()]
    titulo = str(titulo).lower()
    candidato = difflib.get_close_matches(titulo, titulos, n=1, cutoff=0.6)
    return candidato[0] if candidato else None

In [245]:
encontrar_filme('titanic')

'Titanic'

In [ ]:
# def responder(pergunta):
#     # Predição de intenção
#     intent = modelo.predict([pergunta])[0]

#     # Combina todas as intenções conhecidas: iniciais + aprendidas
#     todas_intencoes = intencoes_iniciais + base_aprendida["intencao"].tolist()

#     # Se a intenção estiver nas pré-definidas → usa resposta padrão
#     if intent in intencoes_iniciais:
#         index = intencoes_iniciais.index(intent)
#         return respostas_iniciais[index], intent

#     # Se a intenção estiver entre as aprendidas → usa resposta salva no CSV
#     elif intent in base_aprendida["intencao"].values:
#         resposta = base_aprendida.loc[base_aprendida["intencao"] == intent, "resposta"].values[0]
#         return resposta, intent

#     # Caso o modelo preveja algo desconhecido → pede aprendizado
#     return None, intent


In [253]:
def responder(pergunta):
    print(pergunta)
    # Predizer intenção
    intent = modelo.predict([pergunta])[0]
    print(intent)
    # Respostas baseadas na intenção
    if pergunta.strip() != '':
        # Verifica se a intenção tem resposta no dataset
        if intent == "melhor_nota":
            filme = df.loc[df['rating_imdb'].idxmax()]
            return f"O filme mais bem avaliado é '{filme['title']}' com nota {filme['rating_imdb']} no IMDb.", intent

        elif intent == "maior_bilheteria":
            filme = df.loc[df['gross_world_wide'].idxmax()]
            return f"O filme com maior bilheteria mundial é '{filme['title']}' com ${filme['gross_world_wide']:,} arrecadados.", intent

        elif intent == "diretor_filme":
            palavras = pergunta.split()
            for p in palavras:
                filme = encontrar_filme(p)
                if filme:
                    diretor = df.loc[df['title'] == filme, 'director'].values[0]
                    return f"O diretor de '{filme}' é {diretor}.", intent
            return "Não encontrei o filme na base de dados.", intent
        
        elif intent == "quantidade_oscar_filme":
            palavras = pergunta.split()
            for p in palavras:
                filme = encontrar_filme(p)
                if filme:
                    oscars = df.loc[df['title'] == filme, 'oscar'].values[0]
                    return f"O filme '{filme}' ganhou {oscars} Oscars.", intent
            return "Não encontrei informações sobre Oscars para esse filme.", intent
        
        elif intent == "genero_filme":
            palavras = pergunta.split()
            for p in palavras:
                filme = encontrar_filme(p)
                if filme:
                    genero = df.loc[df['title'] == filme, 'genre'].values[0]
                    return f"O gênero do filme '{filme}' é {genero}.", intent
            return "Não encontrei o gênero do filme.", intent
        
        elif intent == "mais_indicacoes":
            filme = df.loc[df['nomination'].idxmax()]
            return f"O filme com mais indicações ao Oscar é '{filme['title']}' com {filme['nomination']} indicações.", intent
        
        elif intent == "maior_duracao":
            filme = df.loc[df['duration_min'].idxmax()]
            return f"O filme mais longo é '{filme['title']}' com duração de {filme['duration_min']} minutos.", intent
        
        elif intent == "filme_recente":
            filme = df.loc[df['year'].idxmax()]
            return f"O filme mais recente é '{filme['title']}', lançado em {filme['year']}.", intent      

        ########### INTENTS NOVAS ###########
        elif intent == "menor_bilheteria":
            filme = df.loc[df['gross_world_wide'].idxmin()]
            return f"O filme com menor bilheteria mundial é '{filme['title']}' com ${filme['gross_world_wide']:,} arrecadados.", intent
        
        elif intent == "pior_nota":
            filme = df.loc[df['rating_imdb'].idxmin()]
            return f"O filme menos bem avaliado é '{filme['title']}' com nota {filme['rating_imdb']} no IMDb.", intent
        
        elif intent == "diretor_melhor_avaliado":
            mlr_diretor = df.loc[df.groupby('director')['rating_imdb'].idxmax()]
            return f"O diretor de '{mlr_diretor['title']}' é {mlr_diretor['director']}.", intent


        # Caso o modelo não tenha aprendido a responder, retorna None
        else: return None, intent
    
    return None, intent

In [252]:
while True:
    pergunta_usuario = input("\nSua pergunta: ")

    if pergunta_usuario.lower() == "sair":
        print("Saindo e salvando o aprendizado...")
        break

    print(intent)

    resposta, intent = responder(pergunta_usuario)

    if resposta:
        print(resposta)
    else:
        print("Não sei responder isso ainda 😢")
        nova_intencao = input("Digite uma intenção para essa pergunta: ")
        nova_resposta = input("Digite a resposta correta: ")

        # Salva no CSV para aprendizado futuro
        salvar_nova_pergunta(pergunta_usuario, nova_resposta, nova_intencao)

        # Treina o modelo novamente com a nova pergunta
        perguntas_treinamento = perguntas_iniciais + base_aprendida["pergunta"].tolist()
        intencoes_treinamento = intencoes_iniciais + base_aprendida["intencao"].tolist()
        modelo.fit(perguntas_treinamento, intencoes_treinamento)

        print("Obrigado! Aprendi essa nova informação 🎓")

diretor_filme
responda como se voce fosse clovis de barros: quem ganhou o oscar em 1991?
diretor_filme
O diretor de 'Persona' é Ingmar Bergman.
diretor_filme

quantidade_oscar_filme
Não sei responder isso ainda 😢
Obrigado! Aprendi essa nova informação 🎓
quantidade_oscar_filme
saqir
quantidade_oscar_filme
O filme 'Hair' ganhou 0 Oscars.
Saindo e salvando o aprendizado...
